# Instructions

## Setup

1. Rename the file '.env_example' to '.env'.
1. Enter any necessary variables, such as host groups and credentials. (If a variable does not apply, leave it blank.)
1. Run the cells below sequentially.

## Notes

**Mac users:**
* You might get this error: `Unable to revert mtime: /Library/Fonts`
* The solution is to install the *libmagic* brew: `brew install libmagic`
* Reference: [Python MacOS Error: Unable to revert mtime: /Library/Fonts](https://stackoverflow.com/questions/62279920/python-macos-error-unable-to-revert-mtime-library-fonts)

# Create OS Environment Variables

In [ ]:
from netmanage.setup import *
from dotenv import load_dotenv
load_dotenv(override=True)

# Select and Run Collectors

## Select the hostgroups to run the collectors on

**Note:**

* If running this in a miniconda environment and the widget does not display, run the command below.
* See: https://stackoverflow.com/questions/36351109/ipython-notebook-ipywidgets-does-not-show

```
jupyter nbextension enable --py --sys-prefix widgetsnbextension
```

In [ ]:
private_data_dir = os.path.expanduser(os.environ['private_data_directory'])
hostgroup_select = select_hostgroups(collector_select, hostgroup_select, private_data_dir)
for key in sorted(hostgroup_select.keys()):
    print(key.upper())
    display(widgets.GridBox(hostgroup_select[key],
                            layout=widgets.Layout(grid_template_columns='repeat(3, 200px)')))
    print('\n')

## Select the collectors to run

In [ ]:
collector_select = select_collectors(collector_select, hostgroup_select)
for key in sorted(collector_select.keys()):
    print(key.upper())
    display(widgets.GridBox(collector_select[key],
                            layout=widgets.Layout(grid_template_columns='repeat(3, 250px)')))
    print('\n')

## Run Collectors

In [ ]:
%%time

load_dotenv(override=True)

nest_asyncio.apply()  # See: https://tinyurl.com/2whp5rpu

# Reload imported modules (used for debugging)
from netmanage import run_collectors as rc
from netmanage.collectors import cisco_asa_collectors as cac
from netmanage.collectors import cisco_ios_collectors as cic
from netmanage.collectors import dnac_collectors as dnc
from netmanage.collectors import f5_collectors as f5c
from netmanage.collectors import infoblox_nios_collectors as nc
from netmanage.collectors import meraki_collectors as mc
from netmanage.collectors import netbox_collectors as nbc
from netmanage.collectors import palo_alto_collectors as pac
from netmanage.collectors import solarwinds_collectors as swc
from netmanage.parsers import cisco_asa_parsers as cap
from netmanage.parsers import cisco_ios_parsers as cip
from netmanage.parsers import cisco_nxos_parsers as cnp
from netmanage.parsers import palo_alto_parsers as pap
from netmanage.helpers import helpers as hp
importlib.reload(cac)
importlib.reload(cic)
importlib.reload(dnc)
importlib.reload(f5c)
importlib.reload(hp)
importlib.reload(rc)
importlib.reload(nc)
importlib.reload(mc)
importlib.reload(nbc)
importlib.reload(pac)
importlib.reload(swc)
importlib.reload(cap)
importlib.reload(cip)
importlib.reload(cnp)
importlib.reload(pap)

# Create a dataframe of collectors to run
print('COLLECTORS TO RUN\n')
df_collectors = create_collectors_df(collector_select, hostgroup_select)
display(df_collectors.style.hide(axis="index"))

# Set the timestamp so it will be consistent for all collectors
timestamp = dt.datetime.now()
timestamp = timestamp.strftime('%Y-%m-%d_%H%M')

# If palo_alto_serials exists and panorama_get_managed_devices is not in the list of
# collectors, then run the panorama_get_managed_devices collector and add it to the
# database.
palo_alto_serials = list(filter(None, os.environ["palo_alto_serials"].split(",")))
palo_alto_serials = [_.strip() for _ in palo_alto_serials]
if palo_alto_serials and 'panorama_get_managed_devices' not in df_collectors['collector'].to_list():
    hostgroups = set(df_collectors[df_collectors['ansible_os'] == 'paloaltonetworks.panos']['hostgroup'].tolist())
    palo_alto_username = os.environ["palo_alto_username"]
    palo_alto_password = os.environ["palo_alto_password"]
    for hostgroup in hostgroups:
        result = pac.panorama_get_managed_devices(
            os.environ["palo_alto_username"],
            os.environ["palo_alto_password"],
            hostgroup,
            os.path.expanduser(os.environ["netmanage_path"].rstrip("/")),
            os.path.expanduser(os.environ["private_data_directory"])
        )
        if not result.empty:
            rc.add_to_db(
                'PANOS_PANORAMA_MANAGED_DEVICES',
                result,
                timestamp,
                f'{os.path.expanduser(os.environ["database_path"])}/{os.environ["database_name"]}',
                method=os.environ["database_method"]
            )

# Execute the collectors
for idx, row in df_collectors.iterrows():
    ansible_os = row['ansible_os']
    hostgroup = row['hostgroup']
    collector = row['collector']
    result = rc.collect(ansible_os,
                        collector,
                        hostgroup,
                        timestamp)
    print(f'\nRESULT: {ansible_os.upper()} {collector.upper()} COLLECTOR\n')
    display(result)

# Run Validators

## Reload imported modules (optional; used for testing)

In [ ]:
from netmanage import run_collectors as rc
from netmanage import validators as vl
from netmanage.collectors import collectors as cl
from netmanage.collectors import meraki_collectors as mc
from netmanage.helpers import helpers as hp

importlib.reload(cl)
importlib.reload(mc)
importlib.reload(hp)
importlib.reload(rc)
importlib.reload(vl)

## F5 Validators

### ```f5_vip_availability```

In [ ]:
df_diff = vl.f5_vip_availability(db_path, 'bigip_vip_availability')
display(df_diff)

### ```f5_pool_availability```

In [ ]:
df_diff = vl.f5_pool_availability(db_path, 'bigip_pool_availability')
display(df_diff)

### ```f5_pool_member_availability```

In [ ]:
df_diff = vl.f5_pool_member_availability(db_path, 'bigip_pool_member_availability')
display(df_diff)

## Meraki Validators

### ```meraki_get_org_device_statuses```

In [ ]:
%%time
df_diff = vl.meraki_device_statuses_availability(db_path, 'meraki_get_org_device_statuses')
if len(df_diff) > 0:
    display(df_diff.sort_values(by='networkId', ascending=True).style.hide(axis="index"))
else:
    display(df_diff)